<a href="https://colab.research.google.com/github/dxspimentel/PUC/blob/main/MVP_Diego_Pimentel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MVP Análise de Dados e Boas Práticas**

Nome: Diego Pimentel <br>
Matrícula: 4052025000258 <br>
Dataset: **Escolas Conectadas**

<br>
--- OBSERVAÇÕES IMPORTANTES ---

Este é um Dataset adaptado, devido a Segurança da Informação e Sigilo da Insituição.

------

# Descrição do Problema

Dados relacionados ao projeto Escolas Conectadas, cujo objetivo é levar internet a escolas com estrutura precária, ou sem estrutra, das regiões mais afastadas das grandas cidades.<br>
Neste trabalho será desenvolida uma solução que facilite a identificação de quais escolas foram de fato entregues, sem pendências, dentro do universo de escolas com obras já contratadas.

# Hipóteses do Problema

Cada escola tem a sua particularidade, com isso trabalha com escopo específico.
O projeto de cada escola é dividido em 3 tipos de obra.
1) Rede interna
2) Rede externa
3) Gerador solar